In [27]:
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget
from nwbwidgets.allen import AllenRasterWidget
from nwbwidgets.allen import AllenPSTHWidget
import pandas as pd
import numpy as np
from helpers import *
from generate import *

# Load the file with the mice data
nwbfiles = []

paths = ['/Users/shrinidhivelan/Desktop/LSENS - Semester Project/AB077_20230531_143839.nwb', 
        '/Users/shrinidhivelan/Desktop/LSENS - Semester Project/AB087_20231017_141901.nwb']
mouse_names = ['AB077_20230531_143839', 'AB087_20231017_141901']

for path in paths:
    io = NWBHDF5IO(path)
    nwbfiles.append(io.read())

io = NWBHDF5IO('/Users/shrinidhivelan/Desktop/LSENS - Semester Project/AB077_20230531_143839.nwb', mode='r')
nwbfile = io.read()
# To do : create a list of mouse names, to be used for the .py file.
mouse_name = 'AB077_20230531_143839'

/Users/shrinidhivelan/miniconda3/envs/Lsens_SP/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.1 because version 1.8.0 is already loaded.
  return func(args[0], **pargs)
/Users/shrinidhivelan/miniconda3/envs/Lsens_SP/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'core' version 2.6.0-alpha because version 2.7.0 is already loaded.
  return func(args[0], **pargs)
/Users/shrinidhivelan/miniconda3/envs/Lsens_SP/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.2.0 because version 0.5.0 is already loaded.
  return func(args[0], **pargs)


### Loading the tables and preprocessing

#### For each line of trials table, we start by testing 10 neurons. In other words, if there is a stimulus (whisker or auditory) at a certain start time then, compute the number of spikes for each trial for each neuron.

In [2]:
units = nwbfile.units.to_dataframe()
trials = nwbfile.trials.to_dataframe()

# Preprocessing s
filtered_units = units[(units['bc_label'] == 'good') & (units['ccf_acronym'].str.contains('[A-Z]'))]
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 10)  # Show all columns


# Some chosen columns
cons_columns  = ["cluster_id", "firing_rate", "ccf_acronym", "ccf_name", "ccf_parent_acronym", "ccf_parent_name", "spike_times"]#, "lick_flag", "lick_time"]
filtered_units = filtered_units[cons_columns]

In [3]:
filtered_units

,cluster_id,firing_rate,ccf_acronym,ccf_name,ccf_parent_acronym,ccf_parent_name,spike_times
id,,,,,,,
1,1,4.97110188,ACAv5,"Anterior cingulate area, ventral part, layer 5",ACAv,"Anterior cingulate area, ventral part","[0.479889, 1.1120556666666668, 2.586543, 4.527..."
3,3,0.14528229,ACAv5,"Anterior cingulate area, ventral part, layer 5",ACAv,"Anterior cingulate area, ventral part","[7.349212333333333, 36.45573866666666, 56.7888..."
4,4,3.30061586,ACAv5,"Anterior cingulate area, ventral part, layer 5",ACAv,"Anterior cingulate area, ventral part","[0.680589, 2.2582096666666667, 2.8338096666666..."
6,6,0.71495552,ACAv5,"Anterior cingulate area, ventral part, layer 5",ACAv,"Anterior cingulate area, ventral part","[0.17062233333333332, 15.114042666666668, 22.1..."
7,7,4.93621331,ACAv5,"Anterior cingulate area, ventral part, layer 5",ACAv,"Anterior cingulate area, ventral part","[0.004989000000000021, 0.102989, 0.155489, 0.3..."
...,...,...,...,...,...,...,...
1128,719,0.1002392,SSp-bfd6a,"Primary somatosensory area, barrel field, laye...",SSp-bfd,"Primary somatosensory area, barrel field","[87.81333416852802, 161.4544207068485, 179.951..."
1129,720,0.51082935,SSp-bfd5,"Primary somatosensory area, barrel field, layer 5",SSp-bfd,"Primary somatosensory area, barrel field","[0.05410021605180648, 0.2121312593676502, 0.32..."
1131,722,0.10544643,SSp-bfd2/3,"Primary somatosensory area, barrel field, laye...",SSp-bfd,"Primary somatosensory area, barrel field","[15.397901203225924, 30.19794473615663, 47.709..."


### Whisker &  Stimulation
#### Now iterate in each row of short\_units and create a column with tables to append values for number of spikes for each 

In [4]:
trials['context_background'].unique()

array([nan])

In [6]:
### part 1 : with passive trials
#active_trials = trials[trials['context']] == 'passive'

data = spike_detection(filtered_units, trials, type = 'whisker')
data = spike_detection(data, trials, type = 'auditory')
data = spike_detection(data, trials, type = 'lick_stim', file=nwbfile)

In [ ]:
### part 2 : with active trials
passive_trials = trials[trials['context']] == 'active'

data = spike_detection(filtered_units, trials, type = 'whisker')
data = spike_detection(data, trials, type = 'auditory')
data = spike_detection(data, trials, type = 'lick_stim', file=nwbfile)

In [8]:
# Create a new dataframe 

data['pre'] = 0.2
data['post'] = 0.2
data['Mouse name'] = mouse_name



folder = f'Data/{mouse_name}'
os.makedirs(folder, exist_ok=True)

#### Only problem : arrays present in columns are converted to string of array, so careful while preprocessing ####

###### It is better to use parquet files ######
#data.to_csv(f'{mouse_name}_Selectivity_Dataframe.csv', index=True, header=True)
#data.to_pickle(f'{mouse_name}_Selectivity_Dataframe.pkl')
data.to_parquet(f'{folder}/{mouse_name}_Selectivity_Dataframe.parquet', index=False)